<a href="https://colab.research.google.com/github/joostnjns/Google-Earth-Engine/blob/master/DownloadLandsatSebha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Script using the Google Earth Engine API to download Landsat imagery.**

In [0]:
# Run only at first run: install python API
!pip install earthengine-api

In [185]:
# Import and authenticate GEE
import ee
ee.Authenticate()

# Get other packages
import matplotlib.pyplot as plt
from IPython.display import Image
import pprint
import folium
import time

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/swEHfo2nhVYp0foCyMDXUWE5vKhnMlsB_-e-h0IclZzW84S0RrSq0hU

Successfully saved authorization token.


In [0]:
# Download a Landsat image based on some parameters

# LANDSAT/LT05/C01/T1_SR meaning:
# Look into the ImageCollections of Landsat. 
# In our case use Landsat 5 sensors. 
# USGS processes them in 'collections', with collection 1 (C01) as the only option.
# To get the surface reflectane datasat for collection 1, use T1_SR (tier 1)

In [0]:
### Filter Landsat collection based on dates, AOI and other image metadata

# Set AOI for imagery
aoi = ee.Geometry.Rectangle([14.24, 26.92, 14.58, 27.14])

# Create RGB scene
RGBcollection = collection = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1995-01-01', '1995-01-30').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B3', 'B2', 'B1'])
RGBscene = collection.sort('CLOUD COVER').first()
RGBsceneURL = final.getThumbURL()

# Create full 7-band scene
collection = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1995-01-01', '1995-01-30').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'])
scene = collection.sort('CLOUD COVER').first()
sceneURL = final.getThumbURL()
#sceneURL.getInfo()

# Metadata bands
metaCollection = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1995-01-01', '1995-01-30').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).select(['sr_atmos_opacity', 'sr_cloud_qa', 'pixel_qa', 'radsat_qa'])
atm_opac = metaCollection.sort('CLOUD COVER').first().select(['sr_atmos_opacity'])  # atmospheric opacity
sr_cloud_qa = metaCollection.sort('CLOUD COVER').first().select(['sr_cloud_qa'])  # surface reflectance cloud quality: indicating which pixels might be affected by instrument artifacts or subject to cloud contamination
pixel_qa = metaCollection.sort('CLOUD COVER').first().select(['pixel_qa'])
radsat_qa = metaCollection.sort('CLOUD COVER').first().select(['radsat_qa'])

In [319]:
### Inspect the image.

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, eeImageObject, visParams, name):
  mapID = ee.Image(eeImageObject).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer
# Set visualization parameters.
visParams = {'min':0, 'max':3000}
# Create a folium map object.
myMap = folium.Map(location=[27.05, 14.45], width = 900, height = 700, zoom_start=12)
# Add the elevation model to the map object.
myMap.add_ee_layer(RGBscene, visParams, 'Sebha')
# Add a layer control panel to the map.
myMap.add_child(folium.LayerControl())
# Display the map.
display(myMap)

In [0]:
"""
# If you just want to plot the image
#sceneUrl = scene.getThumbUrl()
#Image(url=sceneUrl)
"""

"""
# One way of using folium
# only get grey values?
map = folium.Map(location=[27.05, 14.45], zoom_start=10, height=500)
folium.raster_layers.ImageOverlay(
    image=RGBsceneURL,
    bounds=[[26.92,14.24],[27.14, 14.58]],
    colormap=lambda x: (1.4, 1.2, 1, 1),#R,G,B,alpha
).add_to(map)
map.add_child(folium.LayerControl())
map
"""

"""# Another way of using follium
mapid = final.getMapId
m = folium.Map(location=[27.05, 14.45], tiles = "Stamen Terrain", width = '30%', height = '50%', zoom_start = 12)
m.add_child(folium.raster_layers.ImageOverlay(first, opacity=.7, bounds=[[26.92, 14.24], [27.14, 14.58]], colormap='orange'))
m """

In [326]:
# Export the image, specifying scale and region.
##Export.image.toDrive({image: scene, description: 'imageToDriveExample', scale: 30, region: aoi});

task_config = {
    'scale': 30,
    'fileFormat': 'GeoTIFF',
    'skipEmptyTiles': True,
    'folder': 'Sebha1995'
    }

files = ['sebha_1995_rgb', 'sebha_1995_full']
filenames = [RGBscene, scene]

for i in range(1,len(files)):
  task = ee.batch.Export.image.toDrive(filenames[i], files[i], **task_config)
  task.start()
  # Monitor the task.
  while task.status()['state'] in ['READY', 'RUNNING']:
    print(task.status())
    time.sleep(10)
  else:
    print(task.status())

{'id': '7BOPYYUJE2CSJZD2JBJF5VQC', 'state': 'READY', 'creation_timestamp_ms': 1572539310392, 'update_timestamp_ms': 1572539310392, 'description': 'sebha_1995_full', 'task_type': 'EXPORT_IMAGE'}
{'id': '7BOPYYUJE2CSJZD2JBJF5VQC', 'state': 'RUNNING', 'creation_timestamp_ms': 1572539310392, 'update_timestamp_ms': 1572539315732, 'description': 'sebha_1995_full', 'task_type': 'EXPORT_IMAGE', 'start_timestamp_ms': 1572539315595, 'progress': 0.0}
{'id': '7BOPYYUJE2CSJZD2JBJF5VQC', 'state': 'RUNNING', 'creation_timestamp_ms': 1572539310392, 'update_timestamp_ms': 1572539315732, 'description': 'sebha_1995_full', 'task_type': 'EXPORT_IMAGE', 'start_timestamp_ms': 1572539315595, 'progress': 0.0}
{'id': '7BOPYYUJE2CSJZD2JBJF5VQC', 'state': 'RUNNING', 'creation_timestamp_ms': 1572539310392, 'update_timestamp_ms': 1572539315732, 'description': 'sebha_1995_full', 'task_type': 'EXPORT_IMAGE', 'start_timestamp_ms': 1572539315595, 'progress': 0.0}
{'id': '7BOPYYUJE2CSJZD2JBJF5VQC', 'state': 'RUNNING', '